##Connect Google Drive

##Install Yolo

In [1]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.115 🚀 Python-3.11.3 torch-2.2.2 CPU (Apple M2 Max)
Setup complete ✅ (12 CPUs, 32.0 GB RAM, 677.1/926.4 GB disk)


##Training

In [6]:
from ultralytics import YOLO

epochs = 1 #@param {type:"slider", min:100, max:1000, step:10}
training_option = "Train new CNN" #@param ["Train new CNN", "Resume an interrupted Training"]
weight = 'yolov8m.pt' #@param["yolov8n.pt","yolov8s.pt","yolov8m.pt","yolov8l.pt", "yolov8x"]
batch = 1024 #@param {type:"slider", min:1, max:64, step:1}

model = YOLO(weight)
# data =  "/content/gdrive/MyDrive/"+folder+"/training/training_dataset/data.yaml"
data =  "training/training_dataset/data.yaml"
# project = "/content/gdrive/MyDrive/"+folder+"/training/training_results/"
project = "training/training_results"
# model_folder = "/content/gdrive/MyDrive/"+folder+"/training/training_results/train/weights/last.pt"
model_folder = "training/training_results/train/weights/last.pt"

In [7]:
print(type(model))

<class 'ultralytics.models.yolo.model.YOLO'>


In [ ]:
if training_option == 'Train new CNN':
  model.train(data=data, epochs=epochs, imgsz=512, batch=batch, cache=True, patience=50, project=project)
else:
  model.train(data=data, epochs=epochs, imgsz=512, batch=batch, cache=True, patience=50, project=project, resume=True, model=model_folder)

Ultralytics 8.3.115 🚀 Python-3.11.3 torch-2.2.2 CPU (Apple M2 Max)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=training/training_dataset/data.yaml, epochs=1, time=None, patience=50, batch=1024, imgsz=512, save=True, save_period=-1, cache=True, device=None, workers=8, project=training/training_results, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=Tru

train: Scanning /Users/patebel/Code/crosswalk-detection/training/training_dataset/train/labels.cache... 1070 images, 23 backgrounds, 0 corrupt: 100%|██████████| 1070/1070 [00:00<?, ?it/s]

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



train: Caching images (0.8GB RAM): 100%|██████████| 1070/1070 [00:00<00:00, 3528.01it/s]

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 207.6±10.8 MB/s, size: 44.7 KB)


...

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



val: Caching images (0.2GB RAM): 100%|██████████| 224/224 [00:00<00:00, 4727.05it/s]

Plotting labels to training/training_results/train8/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.008), 83 bias(decay=0.0)
Image sizes 512 train, 512 val
Using 0 dataloader workers
Logging results to training/training_results/train8
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/2 [00:00<?, ?it/s]

## Metrics

In [ ]:
data_test = "/test/test_dataset/data.yaml"
metrics = model.val(data=data_test)

AttributeError: ignored

In [5]:
#Printing result to confront
from PIL import Image

precision_training = '/training/training_results/train/F1_curve.png'
precision_test = '/training/training_results/val/F1_curve.png'

im1 = Image.open(precision_training)
im2 = Image.open(precision_test)

print("F1 Training")
display(im1)
print("F1 Test")
display(im2)

FileNotFoundError: [Errno 2] No such file or directory: '/training/training_results/train/F1_curve.png'

##Detection

In [ ]:
import glob
import os
import shutil
import numpy as np
import cv2 as cv
# from google.colab.patches import cv2_imshow
import cv2
from matplotlib import pyplot as plt

#Bounding Box Merge Algorithm
def merge_bounding_boxes(bounding_boxes,image_width,image_height):
  [x,y] = [image_height,image_width]
  [w,h] = [0,0]
  for i in range(0,bounding_boxes.shape[0]):
    if(bounding_boxes.ndim == 1):
      center_x_1 = int(bounding_boxes[0]*image_width)
      center_y_1 = int(bounding_boxes[1]*image_height)
      width_1 = int(bounding_boxes[2]*image_width)
      height_1 = int(bounding_boxes[3]*image_height)
      x = int(center_x_1 - (width_1/2))
      y = int(center_y_1 - (height_1/2))
      w = int(center_x_1 + (width_1/2))
      h = int(center_y_1 + (height_1/2))
    else:
      center_x_1 = int(bounding_boxes[i,0]*image_width)
      center_y_1 = int(bounding_boxes[i,1]*image_height)
      width_1 = int(bounding_boxes[i,2]*image_width)
      height_1 = int(bounding_boxes[i,3]*image_height)
      vert_x = int(center_x_1 - (width_1/2))
      vert_y = int(center_y_1 - (height_1/2))
      vert_w = int(center_x_1 + (width_1/2))
      vert_h = int(center_y_1 + (height_1/2))
      if(vert_x < x):
        x = vert_x
      if(vert_y < y):
        y = vert_y
      if(vert_w > w):
        w = vert_w
      if(vert_h > h):
        h = vert_h
    return np.array([x,y,w,h])

In [ ]:
!yolo task=detect mode=predict model="/content/gdrive/MyDrive/$folder/training/training_results/train/weights/best.pt" source="/content/gdrive/MyDrive/$folder/detection/dataset/" imgsz=512 save=False save_txt=True save_conf=True

In [ ]:
path = '/runs/detect/predict/labels/'
img_path = '/detection/dataset/'
save_path = '/runs/detect/predict/'

for fname in os.listdir(save_path):
  if fname.endswith('.jpg'):
    ("The path already contains images, skipping this part.")
    break
else:
  from distutils.dir_util import copy_tree
  copy_tree(img_path, save_path)
  print("Path Copied!")

In [ ]:
for filename in glob.glob(os.path.join(path, '*.txt')):
  with open(os.path.join(os.getcwd(),filename), 'r') as f:
    bounding_boxes = None
    confidences = None
    image_name = os.path.basename(filename).replace('txt', 'jpg')
    for line in f:
      cl, label_x, label_y, label_w, label_h, conf = line.split(' ')
      b = float(conf)
      a = np.array([float(label_x),float(label_y),float(label_w),float(label_h)])
      bounding_boxes = (np.vstack((bounding_boxes, a)) if (bounding_boxes is not None) else a)
      confidences = (np.vstack((confidences, b)) if (confidences is not None) else b)
    #Image Elaboration
    image = cv.imread(img_path+image_name)
    [image_height, image_width, levels] = image.shape
    conf_max = np.amax(confidences)
    [x,y,w,h] = merge_bounding_boxes(bounding_boxes, image_width, image_height)
    #Create Rect
    cv.rectangle(image, (x,y), (w,h), (0,0,255), 4)
    cv.putText(image, 'crosswalk ' + "%.2f" % conf_max, (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
    #Saving Image
    print('Saving ' + image_name)
    cv.imwrite(save_path+image_name, image)

##Intersection Over Union (IoU)

In [ ]:
#INTERESCTION OVER UNION (IOU)
def compute_iou(box1, box2):
  #DATA: box1 = [L1, T1, R1, B1], box2 = [L2, T2, R2, B2]
  L_inter = max(box1[0], box2[0])
  T_inter = max(box1[1], box2[1])
  R_inter = min(box1[2], box2[2])
  B_inter = min(box1[3], box2[3])
  if(R_inter < L_inter) or (B_inter < T_inter):
    return 0
  A_inter = (R_inter - L_inter) * (B_inter - T_inter)
  A1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
  A2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
  A_union = A1 + A2 - A_inter
  iou = A_inter/A_union
  return iou

#READING DATA AND CALCULATING IOU
dataset_path = '/iou/dataset/'
detection_path = '/runs/detect/predict/labels/'

iou_array = None

for filename_1 in glob.glob(os.path.join(dataset_path, '*.txt')):
  with open(os.path.join(os.getcwd(),filename_1), 'r') as f:
    name_1 = os.path.basename(filename_1)
    for filename_2 in glob.glob(os.path.join(detection_path, '*.txt')):
      name_2 = os.path.basename(filename_2)
      with open(os.path.join(os.getcwd(),filename_2), 'r') as g:
        if(name_1 == name_2):
          image_name = os.path.basename(filename_2).replace('txt', 'jpg')
          bounding_boxes_1 = None
          bounding_boxes_2 = None
          for line in f:
            cl_1, label_x_1, label_y_1, label_w_1, label_h_1 = line.split(' ')
            a_1 = np.array([float(label_x_1),float(label_y_1),float(label_w_1),float(label_h_1)])
            bounding_boxes_1 = (np.vstack((bounding_boxes_1, a_1)) if (bounding_boxes_1 is not None) else a_1)
          for line in g:
            cl_2, label_x_2, label_y_2, label_w_2, label_h_2, conf_2 = line.split(' ')
            a_2 = np.array([float(label_x_2),float(label_y_2),float(label_w_2),float(label_h_2)])
            bounding_boxes_2 = (np.vstack((bounding_boxes_2, a_2)) if (bounding_boxes_2 is not None) else a_2)
          image = cv.imread(dataset_path+image_name)
          [image_height, image_width, levels] = image.shape
          merge_box_1 = merge_bounding_boxes(bounding_boxes_1, image_width, image_height)
          merge_box_2 = merge_bounding_boxes(bounding_boxes_2, image_width, image_height)
          iou = compute_iou(merge_box_1, merge_box_2)
          iou_array = (np.vstack((iou_array, iou)) if (iou_array is not None) else iou)
        else:
          continue
print('IoU: ' + str(iou_array.mean())[:4])

IoU: 0.63


##Video Detection

In [ ]:
video = 'video.mp4' #@param {type:"string"}
!yolo task=detect mode=predict model="/content/gdrive/MyDrive/$folder/training/training_results/train/weights/best.pt" source="/content/gdrive/MyDrive/$folder/detection/video/$video" imgsz=512 save_txt=True save_conf=True project="video_results/" save=False

In [ ]:
#CREATING INFERENCE ON VIDEO
import glob
import os
import shutil
import numpy as np
import cv2 as cv
# from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt

path = '/video_results/predict/labels/'
video_path = '/detection/video/'
save_path = '/video_results/'
frames_path = '/video_results/predict/frames/'
video_name = video.replace('.mp4', '')

print('Reading Video: '+video_path+video)
v = cv.VideoCapture(video_path+video)
frame_count = int(v.get(cv.CAP_PROP_FRAME_COUNT))
fps = v.get(cv.CAP_PROP_FPS)
video_w = int(v.get(cv.CAP_PROP_FRAME_WIDTH))
video_h = int(v.get(cv.CAP_PROP_FRAME_HEIGHT))
print('Video Info' + '  FPS: '+str(int(fps))+' Width: '+str(video_w)+' Height: '+str(video_h))

frames = None
for filename in glob.glob(os.path.join(path, '*.txt')):
  with open(os.path.join(os.getcwd(),filename), 'r') as f:
    bounding_boxes = None
    confidences = None
    #Saves the Frame number in an array
    frame_num = int(os.path.basename(filename).replace('.txt', '').replace(video_name +'_',''))
    frames = (np.vstack((frames, frame_num)) if (frames is not None) else frame_num)
    for line in f:
      cl, label_x, label_y, label_w, label_h, conf = line.split(' ')
      b = float(conf)
      a = np.array([float(label_x),float(label_y),float(label_w),float(label_h)])
      bounding_boxes = (np.vstack((bounding_boxes, a)) if (bounding_boxes is not None) else a)
      confidences = (np.vstack((confidences, b)) if (confidences is not None) else b)
    conf_max = np.amax(confidences)
    #Get Frames from Video
    v.set(cv.CAP_PROP_POS_FRAMES, frame_num)
    res, image = v.read()
    #cv2_imshow(image)
    image_height = video_h
    image_width = video_w
    [x,y,w,h] = merge_bounding_boxes(bounding_boxes, image_width, image_height)
    cv.rectangle(image, (x,y), (w,h), (0,0,255), 4)
    cv.putText(image, 'crosswalk ' + "%.2f" % conf_max, (x, y+20), cv.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
    if image is None:
      continue
    cv.imwrite(frames_path+str(frame_num)+'.jpg', image)
v.release()
print("Frame Creation Complete. Run the following code to create video and clear frames from disk.")

In [ ]:
#SAVING NEW VIDEO INFERENCES
cap = cv.VideoCapture(video_path+video)
vid_writer = cv.VideoWriter(save_path+video, cv.VideoWriter_fourcc(*'mp4v'), fps, (video_w, video_h))
frame_num = -1;
while (cap.isOpened()):
  frame_num +=1
  ret, frame = cap.read()
  if ret == True:
    if(frame_num in frames):
      frame = cv.imread(frames_path+str(frame_num)+'.jpg')
    vid_writer.write(frame)
  else:
    break
cap.release()
vid_writer.release()

shutil.rmtree(frames_path)
print('Result Saved on '+save_path+video)

#Export in ONNX

In [ ]:
from ultralytics import YOLO
#model = YOLO("/content/gdrive/MyDrive/Crosswalks_Detection/training/training_results/train/weights/best.pt")
#model.export(format="onnx")
!yolo export model="/training/training_results/train/weights/best.pt" format=onnx